In [81]:
import experiment_manager as xpm
from pathlib import Path

import os
from os.path import join

import pandas as pd

import altair as alt

In [96]:
#construct absolute path to result directory

result_path = join(Path(os.getcwd()).resolve().parents[0], 'noniid')

list_of_experiments = xpm.get_list_of_experiments(result_path)
data = list_of_experiments[0].results
del data['hyperparameters']

Loaded  1  Results from  /home/tim/HHI/fl_distill/results/noniid

Hyperparameters: 
 {'dataset': ['cifar10'], 'distill_dataset': ['stl10'], 'net': ['vgg11s'], 'n_clients': [5], 'classes_per_client': [0.1], 'balancedness': [1.0], 'communication_rounds': [5], 'participation_rate': [0.4], 'local_epochs': [3], 'distill_epochs': [3], 'n_distill': [1000], 'batch_size': [128], 'aggregation_mode': ['FAD'], 'distill_mode': ['regular'], 'distill_phase': ['clients'], 'only_linear': [False], 'pretrained': [None], 'save_model': [None], 'log_frequency': [-100], 'log_path': ['noniid/'], 'job_id': [[]], 'finished': [True], 'log_id': [21160]}

Tracked Variables: 
 ['hyperparameters', 'server_val_accuracy', 'distill_loss', 'distill_acc', 'distill_epochs', 'communication_round', 'epochs', 'lr', 'momentum', 'weight_decay']


In [97]:
data['server_val_accuracy'] = data['server_val_accuracy'][:-1]
distill_acc = pd.DataFrame(data['distill_acc'], columns=[f'client{i}'for i, val in enumerate(data['distill_acc'][0])])
distill_acc.index.name = "round"
distill_acc = distill_acc.reset_index().melt('round')
del data['distill_acc']
print(distill_acc)


    round variable   value
0       0  client0  0.1000
1       1  client0  0.1000
2       2  client0  0.1000
3       3  client0  0.1000
4       4  client0  0.1859
5       0  client1  0.1744
6       1  client1  0.1744
7       2  client1  0.1744
8       3  client1  0.1744
9       4  client1  0.1772
10      0  client2  0.1000
11      1  client2  0.1653
12      2  client2  0.1653
13      3  client2  0.2232
14      4  client2  0.2232
15      0  client3  0.1894
16      1  client3  0.1930
17      2  client3  0.1922
18      3  client3  0.1922
19      4  client3  0.1922
20      0  client4  0.1000
21      1  client4  0.1000
22      2  client4  0.1000
23      3  client4  0.1640
24      4  client4  0.1640


In [98]:
data = pd.DataFrame.from_dict(data)
data

,server_val_accuracy,distill_loss,distill_epochs,communication_round,epochs,lr,momentum,weight_decay
0,0.1000,1,1,1,3,0.04,0.9,0.00005
1,0.1075,1,1,2,6,0.04,0.9,0.00005
2,0.1000,1,1,3,9,0.04,0.9,0.00005
3,0.1000,1,1,4,12,0.04,0.9,0.00005
4,0.1706,1,1,5,15,0.04,0.9,0.00005


In [101]:
mean = alt.Chart(distill_acc).mark_line().encode(
    x='round',
    y='mean(value)',
)

band = alt.Chart(distill_acc).mark_errorband(extent='ci').encode(
    x='round',
    y=alt.Y('value', title='Miles/Gallon'),
)

mean + band

alt.LayerChart(...)